# WELCOME TO  LANGCHAIN

In [2]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI()
chat = ChatOpenAI()

# text davinchi model 
a = llm.predict("How many planets are there?")

# chat-gpt-terbo
b = chat.predict("How many planets are there?")

a, b

('\n\nThere are eight planets in our Solar System: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.',
 'As of now, there are eight confirmed planets in our solar system. These include Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. However, there is ongoing debate regarding the classification of Pluto as a planet, as it was reclassified as a "dwarf planet" in 2006 by the International Astronomical Union.')

In [5]:
chat = ChatOpenAI(
    temperature=0.1, # 얼마나 창의점을 하는지(높을수록 더 창의적)
)

In [7]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="You are a geopraphy expert. And you only reply in Italian."),
    AIMessage(content="Ciao, mi chiamo Paolo!"),
    HumanMessage(content="What is the distance between Mexico and Thailand. Also, what is your name?")
]

chat.predict_messages(messages)

AIMessage(content='Ciao! Il mio nome è Paolo. La distanza tra il Messico e la Thailandia è di circa 17.000 chilometri.')

In [3]:
# prompt template
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}. Also, what is your name?")
prompt = template.format(country_a="Mexico", country_b="Thailand")

chat.predict(prompt)

In [14]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geopraphy expert. And you only reply in {language}."),
        ("ai", "Ciao, mi chiamo {name}!"),
        ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?"),
    ]
)

# validate
prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand",
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σου! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 17.000 χιλιόμετρα.')

In [17]:
# output parser
# langchain expression parser
from langchain.schema import BaseOutputParser

class CommanOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",") # strip: 앞 뒤 공백 제거
        return list(map(str.strip, items))

In [22]:
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with \
                a comma separated list of max {max_items} in lower case. Do NOT reply with anything else"),
    ("human", "{question}")
])

prompt = template.format_messages(
    max_items=10,
    # question="What are the planets?"
    question="What are the colors?"
)

result = chat.predict_messages(prompt)

p = CommanOutputParser()
p.parse(result.content)

['red',
 'orange',
 'yellow',
 'green',
 'blue',
 'indigo',
 'violet',
 'black',
 'white',
 'gray']

In [23]:
# |: super magical operator for langchane 
chain = template | chat | CommanOutputParser()
chain.invoke({
    "max_items": 5,
    "question": "What are the pokemons?"
})

# all = chain_one | chain_two 

['pikachu', 'charizard', 'bulbasaur', 'squirtle', 'jigglypuff']

In [35]:
# chain component prompt, chatmodel, outputparser(already used) | retriever, tool
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies \
                for any type of cuisine with easy to find ingredients."),
     "human", "I want to cook {cuisine} food",
])

chef_chain = chef_prompt | chat

In [36]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegeterian chef specialized on making traditional recipies \
                vegetarian. You find alternative ingredients and explain their preparation. \
                You don't radically modify the recipe. If there is no alternative for a food \
                just say you don't know how to replace it"),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

In [38]:
# Langchain method: Runnable map 
final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({"cuisine": "indian"})

AIMessage(content="To make this Chicken Tikka Masala recipe vegetarian, you can replace the boneless chicken with a plant-based alternative such as tofu or tempeh. Here's how you can prepare the alternative ingredients:\n\nAlternative Ingredients:\n- 500g firm tofu or tempeh, cut into bite-sized pieces\n\nInstructions:\n1. If using tofu, drain the tofu and press it between paper towels or a clean kitchen towel to remove excess moisture. Cut the tofu into bite-sized pieces. If using tempeh, simply cut it into bite-sized pieces.\n\n2. In a bowl, combine the plain yogurt, lemon juice, 1 teaspoon of cumin, 1 teaspoon of coriander, turmeric, paprika, and chili powder. Add the tofu or tempeh pieces and mix well. Marinate for at least 1 hour, or overnight in the refrigerator for best results.\n\n3. Heat the vegetable oil in a large pan over medium heat. Add the chopped onion and cook until golden brown, stirring occasionally.\n\n4. Add the minced garlic and grated ginger to the pan. Cook for 

In [40]:
# chain component prompt, chatmodel, outputparser(already used) | retriever, tool
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies \
                for any type of cuisine with easy to find ingredients."),
     "human", "I want to cook {cuisine} food",
])

chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegeterian chef specialized on making traditional recipies \
                vegetarian. You find alternative ingredients and explain their preparation. \
                You don't radically modify the recipe. If there is no alternative for a food \
                just say you don't know how to replace it"),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain
final_chain.invoke({"cuisine": "indian"})

Great! Indian cuisine is known for its rich flavors and aromatic spices. Here's a simple recipe for Chicken Tikka Masala, a popular Indian dish:

Ingredients:
- 500g boneless chicken, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 tablespoons vegetable oil
- 1 large onion, finely chopped
- 3 cloves of garlic, minced
- 1-inch piece of ginger, grated
- 2 teaspoons ground cumin
- 2 teaspoons ground coriander
- 1 teaspoon turmeric powder
- 1 teaspoon paprika
- 1 teaspoon garam masala
- 1 cup tomato puree
- 1 cup heavy cream
- Salt, to taste
- Fresh cilantro, for garnish

Instructions:
1. In a bowl, combine the yogurt, lemon juice, 1 teaspoon of cumin, 1 teaspoon of coriander, turmeric powder, paprika, and salt. Mix well.
2. Add the chicken pieces to the marinade, ensuring they are well coated. Cover and refrigerate for at least 1 hour, or overnight for best results.
3. Preheat your grill or oven to medium-high heat. If using a grill, thread the marinated ch

AIMessageChunk(content="To make a vegetarian version of Chicken Tikka Masala, you can replace the chicken with a plant-based protein such as tofu or paneer. Here's how you can modify the recipe:\n\nIngredients:\n- 500g tofu or paneer, cut into bite-sized pieces\n- 1 cup plain yogurt (use dairy-free yogurt if vegan)\n- 2 tablespoons lemon juice\n- 2 tablespoons vegetable oil\n- 1 large onion, finely chopped\n- 3 cloves of garlic, minced\n- 1-inch piece of ginger, grated\n- 2 teaspoons ground cumin\n- 2 teaspoons ground coriander\n- 1 teaspoon turmeric powder\n- 1 teaspoon paprika\n- 1 teaspoon garam masala\n- 1 cup tomato puree\n- 1 cup coconut cream (or dairy-free heavy cream)\n- Salt, to taste\n- Fresh cilantro, for garnish\n\nInstructions:\n1. In a bowl, combine the yogurt, lemon juice, 1 teaspoon of cumin, 1 teaspoon of coriander, turmeric powder, paprika, and salt. Mix well.\n2. Add the tofu or paneer pieces to the marinade, ensuring they are well coated. Cover and refrigerate for 

# MODEL IO

In [45]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

# t = PromptTemplate.from_template("{country}의 수도가 어디야?")
t = PromptTemplate(
    template="What is the capital of {country}",
    input_variables=["country"],
)

t.format(country="France")

'What is the capital of France'

In [49]:
# FewShotPromptTemplate
examples = [
    {
        "question": "What do you know about France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,   
    },
]

example_template = """
    Huma: {question}
    AI: {answer}
"""

exmaple_prompt = PromptTemplate.from_template(example_template)
# exmaple_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt=exmaple_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}",
    input_variables=["country"],
)

prompt.format(country="Germany")

chain = prompt | chat

chain.invoke({"country": "Germany"})

AI: 
I know this:
Capital: Berlin
Language: German
Food: Bratwurst and Sauerkraut
Currency: Euro

AIMessageChunk(content='AI: \nI know this:\nCapital: Berlin\nLanguage: German\nFood: Bratwurst and Sauerkraut\nCurrency: Euro')

In [56]:
# FewShotChaMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,   
    },
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert, you git short answers."),
    example_prompt,
    ("human", "What do you know about {country}?")
])

chain = final_prompt | chat

chain.invoke({"country": "Germany"})


        I know this:
        Capital: Berlin
        Language: German
        Food: Bratwurst and Sauerkraut
        Currency: Euro
        

AIMessageChunk(content='\n        I know this:\n        Capital: Berlin\n        Language: German\n        Food: Bratwurst and Sauerkraut\n        Currency: Euro\n        ')

In [ ]:
# LengthBasedExampleSelector
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,   
    },
]

example_prompt = PromptTemplate.from_template("human: {question}\nai:{answer}")
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=10,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="human: what do you know abount {country}?",
    input_variables=["country"],
)

chain = prompt | chat

chain.invoke({"country": "Germany"})
